In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE, Isomap, MDS
from sklearn.datasets import fetch_openml
import umap

def visualize_mnist_projections(digit='3', n_images=10, method='pca', n_samples_max=5000, kernel='rbf', gamma=None):
    """
    Projects MNIST digit to 3D using various dimensionality reduction methods.
    
    Parameters:
    -----------
    digit : str
        Which digit to visualize (0-9)
    n_images : int
        Number of sample images to display on the 3D plot
    method : str
        Projection method: 'pca', 'kpca', 'tsne', 'umap', 'isomap', or 'mds'
    n_samples_max : int
        Maximum number of samples to use (for computational efficiency with non-linear methods)
    kernel : str
        Kernel for KernelPCA: 'rbf', 'poly', 'sigmoid', 'cosine', 'linear'
    gamma : float or None
        Kernel coefficient for rbf, poly and sigmoid kernels. If None, uses 1/n_features
    """
    
    # Load MNIST dataset
    print("Loading MNIST dataset...")
    mnist = fetch_openml('mnist_784', version=1, parser='auto')
    X, y = mnist.data, mnist.target
    
    # Convert to numpy arrays and filter for specific digit
    X = np.array(X)
    y = np.array(y)
    
    # Filter only specified digit
    mask = y == digit
    X_digit = X[mask]
    
    print(f"Found {len(X_digit)} samples of digit {digit}")
    
    # Limit samples for computational efficiency
    if len(X_digit) > n_samples_max:
        print(f"Sampling {n_samples_max} random samples for efficiency...")
        sample_indices = np.random.choice(len(X_digit), n_samples_max, replace=False)
        X_digit = X_digit[sample_indices]
    
    # Normalize the data
    X_digit = X_digit / 255.0
    
    # Perform dimensionality reduction
    print(f"Performing {method.upper()} projection...")
    
    if method.lower() == 'pca':
        reducer = PCA(n_components=100)
        X_reduced_full = reducer.fit_transform(X_digit)
        X_projected = X_reduced_full[:, :3]
        
        # Calculate variance info
        variance_3d = np.sum(reducer.explained_variance_ratio_[:3])
        variance_info = f"{variance_3d:.1%} variance explained"
        
        # For variance plot
        cumulative_variance = np.cumsum(reducer.explained_variance_ratio_)
        show_variance_plot = True
        method_display = 'PCA'
        
    elif method.lower() == 'kpca':
        # Kernel PCA with specified kernel
        # Limit samples further for computational efficiency
        if len(X_digit) > 2000:
            print(f"Kernel PCA is computationally expensive. Using only 2000 samples...")
            X_digit = X_digit[:2000]
        
        kpca_params = {
            'n_components': 3,
            'kernel': kernel,
            'random_state': 42,
            'n_jobs': -1
        }
        
        # Set gamma if provided, otherwise use default
        if gamma is not None:
            kpca_params['gamma'] = gamma
        
        reducer = KernelPCA(**kpca_params)
        X_projected = reducer.fit_transform(X_digit)
        
        variance_info = f"{kernel.upper()} kernel"
        show_variance_plot = False
        method_display = f'Kernel PCA ({kernel})'
        
    elif method.lower() == 'tsne':
        reducer = TSNE(n_components=3, random_state=42, perplexity=30, n_iter=1000)
        X_projected = reducer.fit_transform(X_digit)
        variance_info = "Non-linear projection"
        show_variance_plot = False
        method_display = 't-SNE'
        
    elif method.lower() == 'umap':
        reducer = umap.UMAP(n_components=3, random_state=42, n_neighbors=15, min_dist=0.1)
        X_projected = reducer.fit_transform(X_digit)
        variance_info = "Non-linear projection"
        show_variance_plot = False
        method_display = 'UMAP'
        
    elif method.lower() == 'isomap':
        reducer = Isomap(n_components=3, n_neighbors=10)
        X_projected = reducer.fit_transform(X_digit)
        variance_info = "Non-linear manifold projection"
        show_variance_plot = False
        method_display = 'Isomap'
        
    elif method.lower() == 'mds':
        # MDS is computationally expensive, so we limit it further
        if len(X_digit) > 2000:
            print("MDS is computationally expensive. Using only 2000 samples...")
            X_digit = X_digit[:2000]
        reducer = MDS(n_components=3, random_state=42)
        X_projected = reducer.fit_transform(X_digit)
        variance_info = "Distance-preserving projection"
        show_variance_plot = False
        method_display = 'MDS'
        
    else:
        raise ValueError(f"Unknown method: {method}. Choose from 'pca', 'kpca', 'tsne', 'umap', 'isomap', or 'mds'")
    
    # Create subplots with Plotly
    if show_variance_plot:
        fig = make_subplots(
            rows=1, cols=2,
            column_widths=[0.4, 0.6],
            subplot_titles=('Cumulative Explained Variance', 
                           f'3D {method_display} Projection of MNIST Digit {digit}<br>({len(X_digit)} samples, {variance_info})'),
            specs=[[{'type': 'xy'}, {'type': 'scatter3d'}]]
        )
        
        # Plot 1: Cumulative explained variance (PCA only)
        fig.add_trace(
            go.Scatter(
                x=list(range(1, min(101, len(cumulative_variance) + 1))),
                y=cumulative_variance[:100],
                mode='lines',
                name='Cumulative Variance',
                line=dict(color='blue', width=2)
            ),
            row=1, col=1
        )
        
        # Add reference lines
        fig.add_hline(y=0.95, line_dash="dash", line_color="red", 
                      annotation_text="95%", row=1, col=1)
        fig.add_hline(y=0.90, line_dash="dash", line_color="orange", 
                      annotation_text="90%", row=1, col=1)
        
        # Update variance plot layout
        fig.update_xaxes(title_text="Number of Components", row=1, col=1, range=[0, 100])
        fig.update_yaxes(title_text="Cumulative Explained Variance", row=1, col=1)
        
        col_idx = 2
    else:
        fig = go.Figure()
        col_idx = None
    
    # Select random samples to display as images
    n_samples = min(n_images, len(X_digit))
    random_indices = np.random.choice(len(X_digit), n_samples, replace=False)
    
    # Create axis labels
    axis_prefix = method_display.replace(' ', '_').replace('(', '').replace(')', '').upper()
    
    # Plot: 3D projection - all points
    scatter_all = go.Scatter3d(
        x=X_projected[:, 0],
        y=X_projected[:, 1],
        z=X_projected[:, 2],
        mode='markers',
        name=f'All digit {digit}s',
        marker=dict(
            size=2,
            color='blue',
            opacity=0.3
        ),
        hovertemplate=f'{axis_prefix}1: %{{x:.2f}}<br>{axis_prefix}2: %{{y:.2f}}<br>{axis_prefix}3: %{{z:.2f}}<extra></extra>'
    )
    
    # Highlight selected points
    scatter_highlight = go.Scatter3d(
        x=X_projected[random_indices, 0],
        y=X_projected[random_indices, 1],
        z=X_projected[random_indices, 2],
        mode='markers',
        name=f'{n_samples} highlighted samples',
        marker=dict(
            size=8,
            color='red',
            opacity=0.9,
            line=dict(color='black', width=2)
        ),
        hovertemplate=f'Sample %{{text}}<br>{axis_prefix}1: %{{x:.2f}}<br>{axis_prefix}2: %{{y:.2f}}<br>{axis_prefix}3: %{{z:.2f}}<extra></extra>',
        text=[f'{i+1}' for i in range(n_samples)]
    )
    
    if show_variance_plot:
        fig.add_trace(scatter_all, row=1, col=2)
        fig.add_trace(scatter_highlight, row=1, col=2)
        
        # Update 3D plot layout
        fig.update_scenes(
            xaxis_title=f'{axis_prefix}1',
            yaxis_title=f'{axis_prefix}2',
            zaxis_title=f'{axis_prefix}3',
            row=1, col=2
        )
        
        title_text = f"MNIST Digit {digit} - {method_display} Analysis"
    else:
        fig.add_trace(scatter_all)
        fig.add_trace(scatter_highlight)
        
        # Update 3D plot layout
        fig.update_scenes(
            xaxis_title=f'{axis_prefix}1',
            yaxis_title=f'{axis_prefix}2',
            zaxis_title=f'{axis_prefix}3'
        )
        
        title_text = f"MNIST Digit {digit} - {method_display} 3D Projection<br>({len(X_digit)} samples, {variance_info})"
    
    # Update overall layout
    fig.update_layout(
        height=600,
        showlegend=True,
        title_text=title_text,
        title_font_size=16
    )
    
    fig.show()
    
    # Create a matplotlib figure for the sample images
    fig2, axes = plt.subplots(2, (n_samples + 1) // 2, figsize=(15, 6))
    axes = axes.flatten()
    
    for idx, img_idx in enumerate(random_indices):
        img = X_digit[img_idx].reshape(28, 28)
        axes[idx].imshow(img, cmap='gray')
        axes[idx].set_title(f'Sample {idx+1}\n{axis_prefix}1:{X_projected[img_idx, 0]:.1f}', fontsize=9)
        axes[idx].axis('off')
    
    # Hide extra subplots
    for idx in range(len(random_indices), len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle(f'Sample Images from 3D {method_display} Plot', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"\n--- {method_display} Statistics ---")
    print(f"Total samples: {len(X_digit)}")
    print(f"Original dimensions: {X_digit.shape[1]}")
    
    if method.lower() == 'pca':
        print(f"Variance explained by PC1: {reducer.explained_variance_ratio_[0]:.2%}")
        print(f"Variance explained by PC2: {reducer.explained_variance_ratio_[1]:.2%}")
        print(f"Variance explained by PC3: {reducer.explained_variance_ratio_[2]:.2%}")
        print(f"Total variance explained by 3D: {variance_3d:.2%}")
    elif method.lower() == 'kpca':
        print(f"Kernel: {kernel}")
        if gamma is not None:
            print(f"Gamma: {gamma}")
        print(f"Note: Kernel PCA doesn't have variance explained metrics")
    else:
        print(f"Projection type: {variance_info}")
        print(f"Note: Non-linear methods don't have variance explained metrics")
    
    return reducer, X_projected, X_digit


# Example usage:
if __name__ == "__main__":
    print("=" * 60)
    print("Choose your projection method:")
    print("  'pca'    - Principal Component Analysis (linear)")
    print("  'kpca'   - Kernel PCA (non-linear, various kernels)")
    print("            Kernels: 'rbf', 'poly', 'sigmoid', 'cosine', 'linear'")
    print("  'tsne'   - t-SNE (non-linear, preserves local structure)")
    print("  'umap'   - UMAP (non-linear, fast, preserves global structure)")
    print("  'isomap' - Isomap (non-linear manifold learning)")
    print("  'mds'    - Multidimensional Scaling (distance-preserving)")
    print("=" * 60)
    
    # Example: Run with PCA (original method)
    # reducer, X_proj, X_data = visualize_mnist_projections(digit='3', n_images=12, method='pca')
    
    # Example: Run with Kernel PCA - RBF kernel
    #reducer, X_proj, X_data = visualize_mnist_projections(digit='1', n_images=1, method='kpca', kernel='rbf')
    
    # Example: Run with Kernel PCA - Polynomial kernel
    #reducer, X_proj, X_data = visualize_mnist_projections(digit='1', n_images=12, method='kpca', kernel='poly')
    
    # Example: Run with Kernel PCA - Sigmoid kernel
    #reducer, X_proj, X_data = visualize_mnist_projections(digit='7', n_images=12, method='kpca', kernel='sigmoid')
    
    # Example: Run with Kernel PCA - Cosine kernel
    #reducer, X_proj, X_data = visualize_mnist_projections(digit='5', n_images=12, method='kpca', kernel='cosine')
    
    # Example: Run with UMAP (from original)
    reducer, X_proj, X_data = visualize_mnist_projections(digit='1', n_images=1, method='')
    
    # Compare different kernels for the same digit:
    # for kernel in ['rbf', 'poly', 'sigmoid', 'cosine']:
    #     print(f"\n\n{'='*60}\nTesting {kernel.upper()} kernel\n{'='*60}")
    #     reducer, X_proj, X_data = visualize_mnist_projections(
    #         digit='3', n_images=8, method='kpca', kernel=kernel, n_samples_max=2000
    #     )

Choose your projection method:
  'pca'    - Principal Component Analysis (linear)
  'kpca'   - Kernel PCA (non-linear, various kernels)
            Kernels: 'rbf', 'poly', 'sigmoid', 'cosine', 'linear'
  'tsne'   - t-SNE (non-linear, preserves local structure)
  'umap'   - UMAP (non-linear, fast, preserves global structure)
  'isomap' - Isomap (non-linear manifold learning)
  'mds'    - Multidimensional Scaling (distance-preserving)
Loading MNIST dataset...
Found 7877 samples of digit 1
Sampling 5000 random samples for efficiency...
Performing  projection...


ValueError: Unknown method: . Choose from 'pca', 'kpca', 'tsne', 'umap', 'isomap', or 'mds'

In [ ]:
import umap
import json
import base64
from io import BytesIO
from PIL import Image

def image_to_base64_png(image_array):
    """
    Convert a 28x28 numpy array to a base64-encoded PNG data URI.
    
    Parameters:
    -----------
    image_array : np.array
        28x28 grayscale image array (values 0-1)
    
    Returns:
    --------
    str : base64-encoded PNG data URI
    """
    # Convert to 0-255 range
    img_data = (image_array * 255).astype(np.uint8)
    
    # Create PIL Image
    img = Image.fromarray(img_data, mode='L')
    
    # Save to bytes buffer as PNG
    buffer = BytesIO()
    img.save(buffer, format='PNG')
    buffer.seek(0)
    
    # Encode to base64
    img_base64 = base64.b64encode(buffer.read()).decode('utf-8')
    
    # Return as data URI
    return f"data:image/png;base64,{img_base64}"


def generate_projection_json(digit='3', method='pca', n_samples_output=100, n_samples_max=5000, output_file=None):
    """
    Generate JSON file with 3D projection coordinates and base64-encoded images.
    
    Parameters:
    -----------
    digit : str
        Which digit to visualize (0-9)
    method : str
        Projection method: 'pca', 'tsne', 'umap', 'isomap', or 'mds'
    n_samples_output : int
        Number of samples to include in the JSON output
    n_samples_max : int
        Maximum number of samples to use for projection computation
    output_file : str or None
        Output JSON filename. If None, returns the data without saving.
    
    Returns:
    --------
    list : List of dictionaries with id, x, y, z, and image data
    """
    
    # Load MNIST dataset
    print("Loading MNIST dataset...")
    mnist = fetch_openml('mnist_784', version=1, parser='auto')
    X, y = mnist.data, mnist.target
    
    # Convert to numpy arrays and filter for specific digit
    X = np.array(X)
    y = np.array(y)
    
    # Filter only specified digit
    mask = y == digit
    X_digit = X[mask]
    
    print(f"Found {len(X_digit)} samples of digit {digit}")
    
    # Limit samples for computational efficiency
    if len(X_digit) > n_samples_max:
        print(f"Sampling {n_samples_max} random samples for efficiency...")
        sample_indices = np.random.choice(len(X_digit), n_samples_max, replace=False)
        X_digit = X_digit[sample_indices]
    
    # Normalize the data
    X_digit_normalized = X_digit / 255.0
    
    # Perform dimensionality reduction
    print(f"Performing {method.upper()} projection...")
    
    if method.lower() == 'pca':
        reducer = PCA(n_components=3)
        X_projected = reducer.fit_transform(X_digit_normalized)
        
    elif method.lower() == 'tsne':
        reducer = TSNE(n_components=3, random_state=42, perplexity=30, n_iter=1000)
        X_projected = reducer.fit_transform(X_digit_normalized)
        
    elif method.lower() == 'umap':
        reducer = umap.UMAP(n_components=3, random_state=42, n_neighbors=15, min_dist=0.1)
        X_projected = reducer.fit_transform(X_digit_normalized)
        
    elif method.lower() == 'isomap':
        reducer = Isomap(n_components=3, n_neighbors=10)
        X_projected = reducer.fit_transform(X_digit_normalized)
        
    elif method.lower() == 'mds':
        if len(X_digit_normalized) > 2000:
            print("MDS is computationally expensive. Using only 2000 samples...")
            X_digit = X_digit[:2000]
            X_digit_normalized = X_digit_normalized[:2000]
        reducer = MDS(n_components=3, random_state=42)
        X_projected = reducer.fit_transform(X_digit_normalized)
        
    else:
        raise ValueError(f"Unknown method: {method}. Choose from 'pca', 'tsne', 'umap', 'isomap', or 'mds'")
    
    # Select samples for output
    n_output = min(n_samples_output, len(X_digit))
    output_indices = np.random.choice(len(X_digit), n_output, replace=False)
    
    print(f"Generating JSON with {n_output} samples...")
    
    # Create JSON data
    json_data = []
    for idx, sample_idx in enumerate(output_indices):
        # Get image and reshape
        img = X_digit_normalized[sample_idx].reshape(28, 28)
        
        # Convert to base64
        img_base64 = image_to_base64_png(img)
        
        # Create entry
        entry = {
            "id": int(idx),
            "x": float(X_projected[sample_idx, 0]),
            "y": float(X_projected[sample_idx, 1]),
            "z": float(X_projected[sample_idx, 2]),
            "image": img_base64
        }
        json_data.append(entry)
        
        if (idx + 1) % 10 == 0:
            print(f"  Processed {idx + 1}/{n_output} images...")
    
    # Save to file if specified
    if output_file:
        with open(output_file, 'w') as f:
            json.dump(json_data, f, indent=2)
        print(f"\nJSON saved to: {output_file}")
        print(f"File contains {len(json_data)} samples")
    
    print(f"\n--- Generation Complete ---")
    print(f"Method: {method.upper()}")
    print(f"Digit: {digit}")
    print(f"Total entries: {len(json_data)}")
    
    return json_data

In [ ]:
data = generate_projection_json(
    digit='1',
    method='pca',
    n_samples_output=5000,
    output_file='manifoldData.json'
)